<a href="https://colab.research.google.com/github/ab-rohman/Computer_Vision/blob/main/Covid19_Data_Analitycs_%26_Visualizaation_(UAS_BIGDATA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SEPERTI BIASANYA KITA IMPORT LABRARY2 UTAMA YANG AKAN KITA GUNAKAN NANTI

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

MOUNTING THE GDRIVE, BECAUSE I UPLOAD THE COVID-19 DATASET TO MY GDRIVE

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


INSTALLING OPENJDK FOR THE HADOOP ENVIRONTMENT

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

DOWNLOAD THE HADOOP AND SPARK

In [5]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

PySpark isn't on sys.path by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding pyspark to sys.path at runtime. findspark does the latter.

To initialize PySpark, just call

In [8]:
!pip install -q findspark

In [9]:
import findspark
findspark.init()

In [10]:
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml  import Pipeline     
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
  .master("local")\
  .appName("Colab")\
  .config('spark.ui.port', '4050')\
  .getOrCreate()

In [12]:
df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("/content/drive/MyDrive/BigData/covid_19.csv",header=True)

In [13]:
df.printSchema()

root
 |-- SNo: integer (nullable = true)
 |-- ObservationDate: string (nullable = true)
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Last Update: string (nullable = true)
 |-- Confirmed: double (nullable = true)
 |-- Deaths: double (nullable = true)
 |-- Recovered: double (nullable = true)



In [15]:
df.show(10)

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     01/22/2020|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
|  6|     01/22/2020|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0|
|  7|     01/22/2020|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|
|  8|     01/22/2020|       Guizhou|Mainland China

In [17]:
Province = df.select('Province/State').fillna('Unknown')
df = df.fillna({'Province/State':'Unknown'})

In [18]:
df.show(40)

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     01/22/2020|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
|  6|     01/22/2020|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0|
|  7|     01/22/2020|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|
|  8|     01/22/2020|       Guizhou|Mainland China